In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/california-housing-prices/housing.csv


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Load the dataset
data = pd.read_csv('/kaggle/input/california-housing-prices/housing.csv')

# Prepare the data
# Handle missing values
imputer = SimpleImputer(strategy='median')
numerical_cols = data.select_dtypes(include=np.number).columns
data[numerical_cols] = imputer.fit_transform(data[numerical_cols])

# Create income categories for stratification
data['income_cat'] = pd.cut(data['median_income'],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

# Stratified split using Sturge's Rule (for binning target values)
n_bins = int(np.ceil(np.log2(len(data['median_house_value'])) + 1))
data['house_value_cat'] = pd.cut(data['median_house_value'], bins=n_bins, labels=False)

splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in splitter.split(data, data['house_value_cat']):
    train_set = data.loc[train_index]
    test_set = data.loc[test_index]

# Drop unnecessary categorical and stratification columns
for set_ in (train_set, test_set):
    set_.drop(['income_cat', 'house_value_cat','ocean_proximity'], axis=1, inplace=True)

# Separate features and target
train_features = train_set.drop('median_house_value', axis=1)
train_target = train_set['median_house_value'].copy()
test_features = test_set.drop('median_house_value', axis=1)
test_target = test_set['median_house_value'].copy()

# Scale the features
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
test_features_scaled = scaler.transform(test_features)

# Ridge Regression
ridge_model = Ridge(alpha=1.0, random_state=42)  # Adjust alpha as needed
ridge_model.fit(train_features_scaled, train_target)
ridge_predictions = ridge_model.predict(test_features_scaled)

# Lasso Regression
lasso_model = Lasso(alpha=1.0, random_state=42)  # Adjust alpha as needed
lasso_model.fit(train_features_scaled, train_target)
lasso_predictions = lasso_model.predict(test_features_scaled)

# Evaluate the models
ridge_mae = mean_absolute_error(test_target, ridge_predictions)
ridge_mse = mean_squared_error(test_target, ridge_predictions)
ridge_rmse = np.sqrt(ridge_mse)

lasso_mae = mean_absolute_error(test_target, lasso_predictions)
lasso_mse = mean_squared_error(test_target, lasso_predictions)
lasso_rmse = np.sqrt(lasso_mse)

# Print the results
print("Ridge Regression:")
print("MAE:", ridge_mae)
print("MSE:", ridge_mse)
print("RMSE:", ridge_rmse)

print("\nLasso Regression:")
print("MAE:", lasso_mae)
print("MSE:", lasso_mse)
print("RMSE:", lasso_rmse)

# Findings

print("\nFindings:")

if ridge_rmse < lasso_rmse:
    print("Ridge Regression performs slightly better than Lasso Regression in terms of RMSE.")
else:
    print("Lasso Regression performs slightly better than Ridge Regression in terms of RMSE.")

if ridge_mae < lasso_mae:
    print("Ridge Regression performs slightly better than Lasso Regression in terms of MAE.")
else:
    print("Lasso Regression performs slightly better than Ridge Regression in terms of MAE.")

if ridge_mse < lasso_mse:
    print("Ridge Regression performs slightly better than Lasso Regression in terms of MSE.")
else:
    print("Lasso Regression performs slightly better than Ridge Regression in terms of MSE.")

print("The differences in MAE, MSE, and RMSE between Ridge and Lasso are usually small. The choice of model often depends on whether feature selection is desired. Lasso tends to shrink less important feature coefficients to zero, effectively performing feature selection, while Ridge reduces the magnitude of coefficients without eliminating them.")

print("Alpha parameter tuning is important in both models. Tuning the alpha parameter will greatly change results. For example, a larger alpha in Lasso will result in more features being eliminated. A larger alpha in Ridge will result in smaller coefficient values.")

print("Stratified sampling based on median house value bins ensures that both the training and test sets have a similar distribution of house values, which is important for accurate model evaluation.")

Ridge Regression:
MAE: 50765.08076294403
MSE: 4747063660.8474455
RMSE: 68898.93802409037

Lasso Regression:
MAE: 50764.72973655391
MSE: 4746941378.151994
RMSE: 68898.05061213266

Findings:
Lasso Regression performs slightly better than Ridge Regression in terms of RMSE.
Lasso Regression performs slightly better than Ridge Regression in terms of MAE.
Lasso Regression performs slightly better than Ridge Regression in terms of MSE.
The differences in MAE, MSE, and RMSE between Ridge and Lasso are usually small. The choice of model often depends on whether feature selection is desired. Lasso tends to shrink less important feature coefficients to zero, effectively performing feature selection, while Ridge reduces the magnitude of coefficients without eliminating them.
Alpha parameter tuning is important in both models. Tuning the alpha parameter will greatly change results. For example, a larger alpha in Lasso will result in more features being eliminated. A larger alpha in Ridge will resul